In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook 


In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory = False)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats):
  X = df [feats].values
  y = df['prices_amountmin'].values

  model = DecisionTreeRegressor(max_depth=5)

  scores = cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x:str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
df.features

0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
18275    [{"key":"Style","value":["Desert Boots"]},{"ke...
18276    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
18277    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18278    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18279    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 18280, dtype: object

In [0]:
test = {'key':'value'}
test['key']

str(test)

"{'key': 'value'}"

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item ['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
df['features_parsed'].head()

0    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
1    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
2    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
3    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
4    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
Name: features_parsed, dtype: object

In [0]:
keys = set()

df['features_parsed'].map ( lambda x: keys.update(x.keys()))

len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_'+ key
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat [key] = df [False == df[get_name_feat(key)].isnull()].shape[0]/ df.shape[0]*100


In [0]:
{k:v for k,v in keys_stat.items() if v> 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cate'] = df[get_name_feat(key)].factorize()[0]

In [0]:
feats = ['']

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat']
feats += ['feat_sizearm_cate',
 'feat_number of compartments_cate',
 'feat_charger included_cate',
 'feat_capacity_cate',
 'feat_target audience_cate',
 'feat_bridge width_cate',
 'feat_age gender group_cate',
 'feat_case tone_cate',
 'feat_pattern_cate',
 'feat_special features_cate',
 'feat_hood_cate',
 'feat_cushioning_cate',
 'feat_carats_cate',
 'feat_power type_cate',
 'feat_profession_cate',
 'feat_shirt size_cate',
 'feat_manufacturer number_cate',
 'feat_general warranty_cate',
 'feat_diamond clarity_cate']
 
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat


['categories',
 'brand_cat',
 'feat_fabrication',
 'feat_clothing category',
 'feat_location - country',
 'feat_recommended location',
 'feat_catalog',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_certifications and listings',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_closure style_cate',
 'feat_country////region of manufacture_cate',
 'feat_color family_cate',
 'feat_leg_cate',
 'feat_series name_cate',
 'feat_release_cate',
 'feat_labor_cate',
 'feat_item type_cate',
 'feat_lens color family_cate',
 'feat_metal color_cate',
 'feat_motion control_cate',
 'feat_box_cate',
 'feat_resizable_cate',
 'feat_color/finish family_cate',
 'feat_has mercury_cate',
 'feat_sizebridge_cate',
 'feat_crystal_cate',
 'feat_enhanced visibility (reflective)_cate',
 'feat_number of pieces_cate',
 'feat_country/region of manufacture_cate',
 

In [0]:
X= df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
m.fit(X, y)

print (result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names = feats)

(-58.39220212923046, 4.302517569601383)


Weight,Feature
0.2620 ± 0.0072,brand_cat
0.1024 ± 0.0136,feat_material_cat
0.0408 ± 0.0043,feat_gender_cat
0.0196 ± 0.0012,feat_brand_cat
0.0052 ± 0.0006,feat_style_cat
0.0038 ± 0.0004,feat_power type_cate
0.0019 ± 0.0005,feat_diamond clarity_cate
0.0000 ± 0.0000,feat_pattern_cate
0.0000 ± 0.0000,feat_sizearm_cate
0.0000 ± 0.0000,feat_hood_cate


In [0]:
df[df['brand']== 'nike'].features_parsed.sample(5).values

array([{'sport': 'soccer', 'type': 'cleats'},
       {'condition': 'new with box', 'type': 'cleats'},
       {'style': 'running, cross training', 'material': 'synthetic', 'country/region of manufacture': 'vietnam', 'condition': 'new with box'},
       {'style': 'running, cross training, walking', 'condition': 'new without box'},
       {'style': 'running, cross training', 'material': 'synthetic', 'condition': 'new with box'}],
      dtype=object)

In [105]:
!git add matrix_one/Matrix_day5.ipynb

fatal: pathspec 'matrix_one/Matrix_day5.ipynb' did not match any files


In [106]:
ls


data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
!git add matrix_one/Matrix_day5.ipynb

In [108]:
!git commit -m "day 5 matrix"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@bd7a86d87709.(none)')


In [0]:
!git config --global user.email "marta.abramczuk@gmail.com"
!git config --global user.name "Marta"

In [111]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date
